# Liver Model

## Import some things

In [1]:
def return_sys_path():
    path = '.'
    for _ in range(5):
        if 'Deep_Learning' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path
def return_data_path():
    path = '.'
    for _ in range(5):
        if 'Data' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path

In [2]:
import os, sys
sys.path.append(return_sys_path())
from Deep_Learning.Base_Deeplearning_Code.Data_Generators.TFRecord_to_Dataset_Generator import *
from Deep_Learning.Base_Deeplearning_Code.Callbacks.TF2_Callbacks import Add_Images_and_LR
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from Deep_Learning.Base_Deeplearning_Code.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image
import tensorflow as tf
import tensorflow.keras.backend as K

In [ ]:
%matplotlib inline

In [ ]:
%load_ext tensorboard

In [3]:
base = return_sys_path()
data_path = os.path.join(return_data_path(),'Data','Niftii_Arrays','Records')
train_path = [os.path.join(data_path,'Train')]
validation_path = [os.path.join(data_path,'Validation')]
test_path = os.path.join(data_path,'Test')
model_path = os.path.join(base,'Models')
if not os.path.exists(model_path):
    os.makedirs(model_path)

## We now need some image processors...

#### We will ensure that the images are 256 x 256 (downsampled for speed), normalize them with a mean of 78 and std of 29, add random noise, threshold, and turn into 2 classes

In [4]:
image_size = 128
wanted_keys={'inputs':['image'],'outputs':['annotation']}
image_processors_train = [Expand_Dimensions(axis=-1),
                          Ensure_Image_Proportions(image_size,image_size),
                          Repeat_Channel(repeats=3),
                          Normalize_Images(mean_val=78,std_val=29),
                          Threshold_Images(lower_bound=-3.55,upper_bound=3.55),
                          Return_Outputs(wanted_keys)]
image_processors_validation = [Expand_Dimensions(axis=-1),
                          Ensure_Image_Proportions(image_size,image_size),
                          Repeat_Channel(repeats=3),
                          Normalize_Images(mean_val=78,std_val=29),
                          Threshold_Images(lower_bound=-3.55,upper_bound=3.55),
                          Return_Outputs(wanted_keys)]

In [5]:
batch_size = 5
train_generator = Data_Generator_Class(record_paths=train_path)
validation_generator = Data_Generator_Class(record_paths=validation_path)
image_processors_train += [
            {'shuffle': len(train_generator)}, {'batch': batch_size}, {'repeat'}]
image_processors_validation += [{'repeat'}]
train_generator.compile_data_set(image_processors_train)
validation_generator.compile_data_set(image_processors_validation)


{'shuffle': 2484}
{'batch': 5}
{'repeat'}
{'repeat'}


### Lets visualize one of the examples! With batch_size of 5 and shuffle on, it will be 5 random 2D slices

In [ ]:
x,y = next(iter(train_generator.data_set))
x = x[0].numpy()
y = y[0].numpy()

In [ ]:
plot_Image_Scroll_Bar_Image(x)

In [ ]:
plot_Image_Scroll_Bar_Image(np.argmax(y,axis=-1))

### Alright, lets make our model!

In [7]:
from Deep_Learning.Easy_VGG16_UNet.Keras_Fine_Tune_VGG16_TF2 import VGG_16
from Deep_Learning.Base_Deeplearning_Code.Visualizing_Model.Visualing_Model import visualization_model_class
from tensorflow.keras.optimizers import Adam
from tensorflow.compat.v1 import GPUOptions, ConfigProto, Session
from tensorflow.python.keras.backend import set_session
from Deep_Learning.Base_Deeplearning_Code.Callbacks.TF2_Callbacks import MeanDSC

### This is just a click and play, it builds the VGG16 architecture for you with pre-trained weights

![VGG16_Unet.png](./Deep_Learning/Easy_VGG16_UNet/VGG16_UNet.png)

In [8]:
K.clear_session()
gpu_options = GPUOptions(allow_growth=True)
sess = Session(config=ConfigProto(gpu_options=gpu_options, log_device_placement=False))
set_session(sess)
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [64]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model
model_path = os.path.join(return_sys_path(),'Models')

Layer_3
Layer_2
Layer_1
Layer_0


## These are some tools for visualizing the model

In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

### Lets look at the activations of block1_conv1, the activation, and output

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','block1_conv1_activation','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x[0,...][None,...])

In [ ]:
Visualizing_Class.plot_activations()

## Freezing pre-trained layers

In [9]:
new_model.compile(tf.keras.optimizers.Adam(5e-5), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=[tf.keras.metrics.CategoricalAccuracy(), MeanDSC(num_classes=2)])

In [10]:
def freeze_until_name(model,name):
    set_trainable = False
    for layer in model.layers:
        if layer.name == name:
            set_trainable = True
        layer.trainable = set_trainable
    return model
new_model = freeze_until_name(new_model,'Upsampling0_UNet')

## Checkpoint and run

A checkpoint is a way of assessing the model and determining if we should save it

In [11]:
model_name = 'VGG_16_Model'
model_path_out = os.path.join(model_path,'VGG_16_frozen')
if not os.path.exists(model_path_out):
    os.makedirs(model_path_out)

In [12]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'cp-best.ckpt'), monitor='val_mean_dsc', verbose=1, save_best_only=True,
                              save_weights_only=True, mode='max')

add_images = Add_Images_and_LR(log_dir=model_path_out, validation_data=validation_generator.data_set,
                               number_of_images=len(validation_generator), add_images=True, image_frequency=5,
                               threshold_x=True, target_image_height=128, target_image_width=128)
tensorboard = TensorBoard(log_dir=model_path_out)
callbacks = [checkpoint, tensorboard, add_images]

### Lets view the model real quick

In [ ]:
k = TensorBoard(model_path_out)
k.set_model(new_model)

In [ ]:
%tensorboard --logdir {"./Models"}

### Lets train!

In [13]:
new_model.fit(train_generator.data_set, epochs=5, callbacks=callbacks, steps_per_epoch=len(train_generator),
              validation_data=validation_generator.data_set, validation_steps=len(validation_generator),
              validation_freq=1)

Epoch 1/5
496/496 [==============================] - ETA: 0s - loss: 0.0198 - categorical_accuracy: 0.9928 - mean_dsc: 0.9388- ETA
Epoch 00001: val_mean_dsc improved from -inf to 0.96378, saving model to .\..\..\Models\VGG_16_frozen\cp-best.ckpt
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: .\..\..\Models\VGG_16_frozen\cp-best.ckpt\assets
Writing out images
Writing out image 0
Writing out image 1
Writing out image 2
496/496 [==============================] - 40s 81ms/step - loss: 0.0198 - categorical_accuracy: 0.9928 - mean_dsc: 0.9388 - val_loss: 0.0088 - val_categorical_accuracy: 0.9967 - val_mean_dsc: 0.9638
Epoch 2/5
496/496 [==============================] - ETA: 0s - loss: 0.0067 - categorical_accuracy: 0.9973 - mean_dsc: 0.9773
Epoch 00002: val_mean_dsc improved from 0.96378 to 0.96567, saving model to .\..\..\Models\VGG_16_frozen\cp-best.ckpt
INFO:tensorflow:Assets written to: .\..\..\Models\VGG_16_frozen\cp-

In [ ]:
x,y = next(iter(validation_generator.data_set))

In [ ]:
pred = new_model.predict(x)

In [ ]:
pred[pred<0.5] = 0
pred[pred>0] = 1

In [ ]:
plot_Image_Scroll_Bar_Image(pred[...,1])

# Now lets make our own architecture

### First, lets import some necessary functions

In [14]:
from Deep_Learning.Base_Deeplearning_Code.Models.TF_Keras_Models import my_UNet, Return_Layer_Functions, return_hollow_layers_dict
from Deep_Learning.Base_Deeplearning_Code.Cyclical_Learning_Rate.clr_callback_TF2 import CyclicLR
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

### Define our convolution and strided blocks, strided is used for downsampling

In [15]:
activation = {'activation': 'relu'}
kernel = (3,3)
pool_size = (2,2)
#{'channels': x, 'kernel': (3, 3), 'strides': (1, 1),'activation':activation}
conv_block = lambda x: {'convolution': {'channels': x, 'kernel': (3, 3),
                                        'activation': None, 'strides': (1, 1)}}
pooling_downsampling = {'pooling': {'pooling_type': 'Max',
                                    'pool_size': (2, 2), 'direction': 'Down'}}
pooling_upsampling = {'pooling': {'pool_size': (2, 2), 'direction': 'Up'}}

### Our architecture will have 2 main parts in each layer, an 'Encoding' and a 'Decoding' side, 'Encoding' goes down, and 'Decoding' goes up

![Encoding and Decoding.png](./Deep_Learning/Encoding_and_Decoding.png)

### We will now create our layer dictionary, this tells our UNet what to look like

### If Pooling is left {} it will perform maxpooling and upsampling with pooling()

In [16]:
def get_layers_dict(layers=1, filters=8, max_filters=np.inf, num_conv_blocks=2, num_classes=2,**kwargs):
    lc = Return_Layer_Functions(kernel=(3,3),strides=(1,1),padding='same',batch_norm=True,
                                pooling_type='Max', pool_size=(2,2), bn_before_activation=False)

    block = lc.convolution_layer

    layers_dict = return_hollow_layers_dict(layers)
    pool = (2, 2)
    final_filters = None
    for layer in range(layers - 1):
        layers_dict['Layer_' + str(layer)]['Encoding'] = []
        encoding = []
        for i in range(num_conv_blocks):
            encoding += [block(filters)]
        layers_dict['Layer_' + str(layer)]['Encoding'] += encoding
        layers_dict['Layer_' + str(layer)]['Pooling']['Decoding'] = [lc.upsampling_layer(pool_size=pool),
                                                                     lc.convolution_layer(filters)]
        if filters < max_filters:
            filters = int(filters*2)
        layers_dict['Layer_' + str(layer)]['Pooling']['Encoding'] = lc.convolution_layer(filters, strides=(2,2))
        layers_dict['Layer_' + str(layer)]['Decoding'] = []
        encoding = []
        for i in range(num_conv_blocks):
            encoding += [block(filters)]
        if layer == 0:
            final_filters = filters
        layers_dict['Layer_' + str(layer)]['Decoding'] = encoding
    encoding = []
    for i in range(num_conv_blocks):
        encoding += [block(filters)]
    layers_dict['Base'] = encoding
    final_steps = [lc.convolution_layer(32, batch_norm=True, kernel=(1,1,1),activation='elu'),
                   lc.convolution_layer(num_classes, batch_norm=False, activation='softmax')]
    layers_dict['Final_Steps'] = final_steps
    return layers_dict

In [18]:
layers_dict = {}
lc = Return_Layer_Functions(kernel=(3,3),strides=(1,1),padding='same',batch_norm=True,
                                pooling_type='Max', pool_size=(2,2), bn_before_activation=False)
conv_block = lc.convolution_layer
layers_dict['Layer_0'] = {'Encoding': [conv_block(8),conv_block(8)],
                          'Decoding': [conv_block(16), conv_block(16)],
                          'Pooling':
                              {'Encoding': [lc.convolution_layer(16, strides=(2, 2))],
                               'Decoding': [lc.upsampling_layer(pool_size=(2, 2)),
                                       lc.convolution_layer(16)]
                               }}
layers_dict['Base'] = [conv_block(16), conv_block(16)]
layers_dict['Final_Steps'] =  [lc.convolution_layer(16, batch_norm=True, kernel=(1, 1),activation='elu'),
                               lc.convolution_layer(2, batch_norm=False, activation='softmax')]

In [ ]:
layers_dict['Layer_0'] = {'Encoding': [conv_block(16),activation,conv_block(16),activation],
                          'Decoding': [conv_block(32),activation,conv_block(32),activation],
                          'Pooling':
                              {'Encoding': [pooling_downsampling],
                               'Decoding': [pooling_upsampling]
                               }}
layers_dict['Base'] = [conv_block(32),activation,conv_block(32),activation]
layers_dict['Final_Steps'] = [conv_block(2),{'activation':'softmax'}]

In [19]:
K.clear_session()
gpu_options = GPUOptions(allow_growth=True)
sess = Session(config=ConfigProto(gpu_options=gpu_options, log_device_placement=False))
set_session(sess)
new_model = my_UNet(layers_dict=layers_dict, image_size=(128, 128, 3), is_2D=True).created_model

Layer_0
Base
Layer_0
Final_Steps


### Name your model and define other things! Send a list of strings and it will make a folder path

In [20]:
model_name = 'My_New_Model'
model_path_out = os.path.join(model_path,model_name)
if not os.path.exists(model_path_out):
    os.makedirs(model_path_out)

### Lets look at our model

In [21]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
k = TensorBoard(model_path_out)
k.set_model(new_model)

In [ ]:
%tensorboard --logdir {"./Models"}

### Set a learning rate and loss metric, also add any metrics you want to track

In [22]:
min_lr = 5e-6
max_lr = 1e-3
new_model.compile(tf.keras.optimizers.Adam(5e-5), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=[tf.keras.metrics.CategoricalAccuracy(), MeanDSC(num_classes=2)])

### This is a checkpoint to save the model if it has the highest dice, also to add images

#### We will specify that we want to watch the validation dice, and save the one with the highest value

In [23]:
monitor = 'val_mean_dsc'
mode = 'max'
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'cp-best.ckpt'), monitor=monitor, verbose=1, save_best_only=True,
                             save_weights_only=True, save_freq='epoch', mode=mode)

#### Next, our tensorboard output will add prediction images


#### CyclicLR will allow us to change the learning rate of the model as it runs, and Add_LR_To_Tensorboard will let us view it later

In [24]:
steps_per_epoch = len(train_generator)//3
step_size_factor = 10

cyclic_lrate = CyclicLR(base_lr=min_lr, max_lr=max_lr, step_size=steps_per_epoch * step_size_factor, mode='triangular2')
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'cp-best.ckpt'), monitor=monitor, verbose=1, save_best_only=True,
                              save_weights_only=False, mode='max')

add_images = Add_Images_and_LR(log_dir=model_path_out, validation_data=validation_generator.data_set,
                               number_of_images=len(validation_generator), add_images=True, image_frequency=5,
                               threshold_x=True, target_image_height=128, target_image_width=128)
tensorboard = TensorBoard(log_dir=model_path_out)

### Combine all callbacks

In [25]:
callbacks = [cyclic_lrate, checkpoint, tensorboard, add_images]

In [ ]:
new_model.fit(train_generator.data_set, epochs=5, callbacks=callbacks, steps_per_epoch=len(train_generator),
              validation_data=validation_generator.data_set, validation_steps=len(validation_generator),
              validation_freq=1)

Epoch 1/5
494/496 [============================>.] - ETA: 0s - loss: 0.7898 - categorical_accuracy: 0.6500 - mean_dsc: 0.2064- ETA: 1s - loss: 0.8277 - categorical_accur
Epoch 00001: val_mean_dsc improved from -inf to 0.30857, saving model to .\..\..\Models\My_New_Model\cp-best.ckpt
INFO:tensorflow:Assets written to: .\..\..\Models\My_New_Model\cp-best.ckpt\assets
Writing out images
Writing out image 0
Writing out image 1
Writing out image 2
496/496 [==============================] - 22s 44ms/step - loss: 0.7892 - categorical_accuracy: 0.6505 - mean_dsc: 0.2068 - val_loss: 0.6190 - val_categorical_accuracy: 0.7993 - val_mean_dsc: 0.3086
Epoch 2/5
493/496 [============================>.] - ETA: 0s - loss: 0.5501 - categorical_accuracy: 0.8435 - mean_dsc: 0.4142- ETA: 0s - loss: 0.5534 - categorical_accuracy: 0.8415 - mean_dsc: 0.
Epoch 00002: val_mean_dsc improved from 0.30857 to 0.52687, saving model to .\..\..\Models\My_New_Model\cp-best.ckpt
INFO:tensorflow:Assets written to: .\..\..

In [ ]:
%tensorboard --logdir {"./Models"}